## Laptop Price Prediction (Data Acquisition)

In [53]:
import requests
import urllib3
import time
import json
from bs4 import BeautifulSoup


def fetch_page(start, end):
    url = "https://www.amazon.in/s?k=Laptops&page={i}&crid=YKRYMQ800YC6&sprefix=laptops%2Caps%2C197&ref=nb_sb_noss_1"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }

    data = []
    for i in range(start, end):
        resp = requests.get(url.format(i=i), headers=headers)
        print(f"Fetching Page No.: {i}")
        if resp.status_code == 200:
            data.append({'page': i, 'content': resp.text})
        else:
            print(str(resp.status_code)+' - Error loading the page')
        print("Waiting for 2 seconds")
        time.sleep(2)

    print("Writing to file")
    with open(f'data_{start}_{end}.json', 'w') as f:
        json.dump(data, f, indent=2)

def get_description(path):
    if path.startswith("https://"):
        url = path
    else:
        url = "https://amazon.in" + path
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    }
    resp = requests.get(url, headers=headers)
    if resp.status_code == 200:
        return resp.text
    else:
        print(str(resp.status_code)+' - Error loading the page')
        return None
    
def get_product_detail_from_linkPath(path):
    desc_text = get_description(path)
    sp = BeautifulSoup(desc_text, 'html.parser')
    tbl = sp.find('table', {"id": 'productDetails_techSpec_section_1'})
    res = {}
    for row in tbl.findAll("tr"):
        cells = row.findAll("td")
        headers = row.findAll("th")
        data = {i.text.strip(): j.text.strip().strip('\n').strip('\u200e') for i,j in zip(headers, cells)}
        res = {**res, **data}
    return res

In [67]:
pgs = fetch_page(101,200)
data = json.load(open('data_11_100.json'))
jsonData = []
for page in data:
    soup = BeautifulSoup(page['content'], 'html.parser')
    ptitle = soup.find_all('span', {'class': 'a-size-medium a-color-base a-text-normal'})
    for title in ptitle:
        print("Fetching Data for\t", title.text)
        pgNo = page['page']
        price = title.find_next('span', {'class': 'a-price-whole'}).text
        path = title.find_parent('a')['href']
        jsonData.append({'Product Name': title.text, 'Page Number': pgNo, 'Price': price, **get_product_detail_from_linkPath(path)})
        time.sleep(2)
        with open('product_data_2.json', 'w') as f:
            json.dump(jsonData, f, indent=2)

Fetching Page No.: 101
Waiting for 2 seconds
Fetching Page No.: 102
Waiting for 2 seconds
Fetching Page No.: 103
Waiting for 2 seconds
Fetching Page No.: 104
Waiting for 2 seconds
Fetching Page No.: 105
Waiting for 2 seconds
Fetching Page No.: 106
Waiting for 2 seconds
Fetching Page No.: 107
Waiting for 2 seconds
Fetching Page No.: 108
Waiting for 2 seconds
Fetching Page No.: 109
Waiting for 2 seconds
Fetching Page No.: 110
Waiting for 2 seconds
Fetching Page No.: 111
Waiting for 2 seconds
Fetching Page No.: 112
Waiting for 2 seconds
Fetching Page No.: 113
Waiting for 2 seconds
Fetching Page No.: 114
Waiting for 2 seconds
Fetching Page No.: 115
Waiting for 2 seconds
Fetching Page No.: 116
Waiting for 2 seconds
Fetching Page No.: 117
Waiting for 2 seconds
Fetching Page No.: 118
Waiting for 2 seconds
Fetching Page No.: 119
Waiting for 2 seconds
Fetching Page No.: 120
Waiting for 2 seconds
Fetching Page No.: 121
Waiting for 2 seconds
Fetching Page No.: 122
Waiting for 2 seconds
Fetching P

AttributeError: 'NoneType' object has no attribute 'findAll'

In [1]:
import requests
import urllib3
import time
import json
from bs4 import BeautifulSoup

In [3]:
loadProductData = json.load(open('../data/product_data.json'))

In [4]:
import pandas as pd
df = pd.read_json('../data/product_data.json')

In [5]:
df.shape

(231, 103)

In [6]:
df2 = pd.read_json('../data/product_data_1.json')

In [7]:
df2.shape

(760, 123)

In [8]:
df3 = pd.read_json('../data/product_data_2.json')

In [9]:
df3.shape

(265, 114)

In [10]:
combined_df = pd.concat([df, df2, df3], join="outer")

In [11]:
combined_df.head()

,Product Name,Page Number,Price,Brand,Manufacturer,Series,Colour,Form Factor,Item Height,Item Width,...,Furniture Finish,Assembly Required,Shape,Base Material,Top Material Type,Age Range (Description),Size,Base Type,ASIN,Compatible Devices
0,"Dell 15 Laptop, Intel Core i3-1115G4 Processor...",1,"36,990",Dell,Dell India Pvt Ltd,Vostro 3520,Black,Laptop,53 Centimeters,10.4 Centimeters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Dell 15 Laptop, Intel Core i5-1135G7 Processor...",1,"50,790",Dell,Dell India Pvt Ltd,Vostro 3510,Carbon,laptop,39.5 Centimeters,7.6 Centimeters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"HP Laptop 15s, AMD Ryzen 5 5500U, 15.6-inch (3...",1,"39,800",HP,Hp india private limited,HP Laptop 15s-eq2144AU,Natural Silver,Laptop,18 Millimeters,24.2 Centimeters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"MSI Modern 15, Intel 11th Gen. i5-1155G7, 40CM...",1,"39,990",MSI,MSI,Modern 15 B11M-061IN,Classic Black,Laptop,47.2 Centimeters,7.5 Centimeters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ASUS [SmartChoice] Vivobook 15, Intel Celeron ...",1,"26,990",ASUS,ASUS,Vivobook 15,Transparent Silver,Ultra-Portable,48.5 Centimeters,6.7 Centimeters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
combined_df.to_csv("dataset.csv", index=False)